In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import datetime
import re
from mlxtend.preprocessing import minmax_scaling
import pickle
import math
from sklearn import preprocessing

In [2]:
solar_data_filename = '/Users/brad/Desktop/CS6620/Project/Data/Sample.csv' 
weather_data_filename = '/Users/brad/Desktop/CS6620/Project/Data/weather.csv' 

In [3]:
solarData = pd.read_csv(solar_data_filename, sep=',')

In [4]:
solarData.columns

Index(['Unnamed: 0', 'TIME', 'Solar_power', 'Meter_value'], dtype='object')

In [5]:
solarData.head()

,Unnamed: 0,TIME,Solar_power,Meter_value
0,0,2019-07-11 13:11:55,-5.366741e+08,5.342115
1,1,2019-07-11 13:13:30,-5.365808e+08,5.352649
2,2,2019-07-11 13:13:38,-5.368667e+08,5.352649
3,3,2019-07-11 13:13:45,-5.369603e+08,5.352649
4,4,2019-07-11 13:13:53,-5.371526e+08,5.352649


In [6]:
# times = datetime.datetime.strptime(solarData['TIME'][0], '%Y-%m-%d %H:%M:%S').time()

In [7]:
# Seperate date and time using datetime from pandas 
solarData['Date']= pd.to_datetime(solarData['TIME']).dt.date
solarData['Time']= pd.to_datetime(solarData['TIME']).dt.time

In [8]:
# We Don't need the TIME or meter_value anymore 
solarData = solarData.drop(['TIME', 'Meter_value', 'Unnamed: 0'], axis=1)
solarData.head()

,Solar_power,Date,Time
0,-5.366741e+08,2019-07-11,13:11:55
1,-5.365808e+08,2019-07-11,13:13:30
2,-5.368667e+08,2019-07-11,13:13:38
3,-5.369603e+08,2019-07-11,13:13:45
4,-5.371526e+08,2019-07-11,13:13:53


In [9]:
# According to Shaju, from ASPIRE, negative values are corrupted
solarData['Solar_power'] = solarData['Solar_power'].apply(lambda val : val if val > 0 else math.nan)

In [10]:
solarData.head()

,Solar_power,Date,Time
0,NaN,2019-07-11,13:11:55
1,NaN,2019-07-11,13:13:30
2,NaN,2019-07-11,13:13:38
3,NaN,2019-07-11,13:13:45
4,NaN,2019-07-11,13:13:53


In [11]:
print(solarData['Solar_power'].min())
print(solarData['Solar_power'].max())
print(solarData['Solar_power'].median())
print()
print(solarData['Solar_power'].mean())
print(solarData['Solar_power'].var())
print(solarData['Solar_power'].std())

740.6687727956357
95620.47706058336
17417.057682456667

21697.783441652493
260095452.22527274
16127.475072846113


In [12]:
len( solarData['Solar_power'] )

1375826

In [13]:
len( solarData['Date'] )

1375826

In [14]:
# Plot a histogram of individual generation to get a feel for the distribution
x = np.arange(1375826)
plt.figure()
plt.hist(solarData['Solar_power'], bins=30, edgecolor='black', linewidth=1)
plt.xlabel("power output (watts")
plt.ylabel("Number of occurences")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
df = pd.DataFrame( columns=['Date', 'Hour', 'Generation'])

In [16]:
df.append([solarData['Date'][0], 14, 68735132])

,0,Date,Generation,Hour
0,2019-07-11,NaN,NaN,NaN
1,14,NaN,NaN,NaN
2,68735132,NaN,NaN,NaN


In [17]:
# Now need to accumulate power per hour 
hour = -1
# This will contain a list of tuples where 0th index is date, 1st is hour of day (24 - hour based), 2nd is total generation per hour
date_hour_generation = [] 

# This is how many hours had at least 1 corrupted/garbage value 
numAdjusted = 0

# loop through all the data, 
# NaN values in an hour will be replaced with the average for the non-NaN values in the hour 
i = 0
while i < len(solarData['Solar_power']):
    hourTotal = 0
    numNaN = 0
    numTotal = 0
    hour = solarData['Time'][i].hour
    # get statistics for this single hour 
    while i < len(solarData['Solar_power']) and hour == solarData['Time'][i].hour:
        numTotal += 1 
        if math.isnan(solarData['Solar_power'][i]):
            numNaN += 1 
        else: 
            hourTotal += solarData['Solar_power'][i]
        i += 1
    # end while inner 
    if numNaN == 0:
        date_hour_generation.append((solarData['Date'][i -1], hour, hourTotal))
    elif numNaN == numTotal: # all values of an hour are garbage, skip 
        continue
    else:
        numAdjusted += 1 
        notNaN = numTotal - numNaN
        avg = hourTotal / notNaN
        hourTotal = avg * numTotal 
        date_hour_generation.append((solarData['Date'][i-1], hour, hourTotal))

# end while outer 



In [18]:
print(f'Number of hours adjusted: {numAdjusted}')
print(f'Total number of hours computed: {len(date_hour_generation)}')
print(f'Percentage of hours with some degree of adjustment: {( (numAdjusted / len(date_hour_generation) )* 100 ) :.2f}%')

Number of hours adjusted: 2477
Total number of hours computed: 5601
Percentage of hours with some degree of adjustment: 44.22%


In [19]:
# df is the cumulated generation data frame 
df = pd.DataFrame(date_hour_generation, columns=['Date','Hour', 'Generated'])
df.tail()

,Date,Hour,Generated
5596,2021-03-08,17,883388.264795
5597,2021-03-08,18,535842.293230
5598,2021-03-09,8,528051.160382
5599,2021-03-09,9,557653.033407
5600,2021-03-09,10,407960.580239


In [20]:
df.head()

,Date,Hour,Generated
0,2019-07-11,13,1.900515e+07
1,2019-07-11,14,2.513928e+07
2,2019-07-11,16,2.020907e+07
3,2019-07-11,17,1.596492e+07
4,2019-07-11,18,1.071462e+07


In [21]:
x = np.arange(5601)
plt.figure()
plt.hist(df['Generated'], bins=30, edgecolor='black', linewidth=1)
plt.xlabel("power output")
plt.ylabel("Number of occurences")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
print(df['Generated'].min())
print(df['Generated'].max())
print(df['Generated'].median())
print()
print(df['Generated'].mean())
print(df['Generated'].var())
print(df['Generated'].std())

6801.6095491037695
25803318.484865587
2854785.925450852

4549084.9332789155
21611058285747.918
4648769.545347233


In [23]:
weatherpf = pd.read_csv(weather_data_filename, sep=',')
weatherpf.columns

Index(['date_time', 'station_id', 'rh', 'eto', 'etr', 'rso', 'ea_avg',
       'precip', 'rh_max', 'rh_min', 'rh_tmn', 'rh_tmx', 'td_avg', 'td_max',
       'td_min', 'td_tmn', 'td_tmx', 'co2_avg', 'ppf_avg', 'solarmj',
       'wet_pct', 'airt_avg', 'airt_max', 'airt_min', 'airt_tmn', 'airt_tmx',
       'lwdn_avg', 'lwup_avg', 'pressure', 'swdn_avg', 'swup_avg', 'windd_sd',
       'windd_avg', 'winds_avg', 'winds_max', 'winds_tmx', 'netrad_avg',
       'fan_rmp_avg', 'visibilitymiles', 'batt_volt_min', 'surfacet1_avg',
       'surfacet2_avg', 'visibilitymiles_avg', 'pressurekpasealevel'],
      dtype='object')

In [24]:
weatherpf.head()

,date_time,station_id,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,...,winds_max,winds_tmx,netrad_avg,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel
0,2019-07-11 00:59:59,1279257,30,0.002,0.003,0.0,0.741,0.0,31,29,...,18.119,11JUL2019:00:47:39,-63.37,4252,19.884,11.79,60.8,60.3,19.822,101.6
1,2019-07-11 01:59:59,1279257,31,0.002,0.003,0.0,0.732,0.0,32,30,...,13.198,11JUL2019:01:56:17,-61.05,4263,19.884,11.79,58.5,58.3,19.822,101.6
2,2019-07-11 02:59:59,1279257,32,0.001,0.002,0.0,0.716,0.0,33,30,...,17.448,11JUL2019:02:41:18,-61.61,4275,19.884,11.82,58.3,58.1,19.822,101.6
3,2019-07-11 03:59:59,1279257,32,0.002,0.002,0.0,0.703,0.0,33,30,...,16.553,11JUL2019:03:02:49,-60.99,4241,19.635,11.78,57.6,57.4,19.822,101.6
4,2019-07-11 04:59:59,1279257,32,0.001,0.002,0.0,0.686,0.0,33,30,...,14.987,11JUL2019:04:52:07,-60.79,4231,19.884,11.81,56.8,56.8,19.884,101.7


In [25]:
# we can't feed the date strings into our models, lets just get rid of them
# the data is all from the same weather station. We don't need that either 
toDrop = ['station_id', 'rh_tmn', 'rh_tmx', 'td_tmn', 'td_tmx', 'airt_tmn', 'airt_tmx', 'winds_tmx']
weatherpf = weatherpf.drop(toDrop, axis=1)
weatherpf.head()

,date_time,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,td_avg,...,winds_avg,winds_max,netrad_avg,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel
0,2019-07-11 00:59:59,30,0.002,0.003,0.0,0.741,0.0,31,29,36.9,...,5.369,18.119,-63.37,4252,19.884,11.79,60.8,60.3,19.822,101.6
1,2019-07-11 01:59:59,31,0.002,0.003,0.0,0.732,0.0,32,30,36.5,...,3.803,13.198,-61.05,4263,19.884,11.79,58.5,58.3,19.822,101.6
2,2019-07-11 02:59:59,32,0.001,0.002,0.0,0.716,0.0,33,30,36.0,...,4.698,17.448,-61.61,4275,19.884,11.82,58.3,58.1,19.822,101.6
3,2019-07-11 03:59:59,32,0.002,0.002,0.0,0.703,0.0,33,30,35.4,...,4.250,16.553,-60.99,4241,19.635,11.78,57.6,57.4,19.822,101.6
4,2019-07-11 04:59:59,32,0.001,0.002,0.0,0.686,0.0,33,30,34.9,...,4.026,14.987,-60.79,4231,19.884,11.81,56.8,56.8,19.884,101.7


In [26]:
# Seperate date and time using datetime from pandas 
weatherpf['Date']= pd.to_datetime(weatherpf['date_time']).dt.date
weatherpf['Hour']= pd.to_datetime(weatherpf['date_time'][:]).dt.hour

In [27]:
weatherpf.head()

,date_time,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,td_avg,...,netrad_avg,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel,Date,Hour
0,2019-07-11 00:59:59,30,0.002,0.003,0.0,0.741,0.0,31,29,36.9,...,-63.37,4252,19.884,11.79,60.8,60.3,19.822,101.6,2019-07-11,0
1,2019-07-11 01:59:59,31,0.002,0.003,0.0,0.732,0.0,32,30,36.5,...,-61.05,4263,19.884,11.79,58.5,58.3,19.822,101.6,2019-07-11,1
2,2019-07-11 02:59:59,32,0.001,0.002,0.0,0.716,0.0,33,30,36.0,...,-61.61,4275,19.884,11.82,58.3,58.1,19.822,101.6,2019-07-11,2
3,2019-07-11 03:59:59,32,0.002,0.002,0.0,0.703,0.0,33,30,35.4,...,-60.99,4241,19.635,11.78,57.6,57.4,19.822,101.6,2019-07-11,3
4,2019-07-11 04:59:59,32,0.001,0.002,0.0,0.686,0.0,33,30,34.9,...,-60.79,4231,19.884,11.81,56.8,56.8,19.884,101.7,2019-07-11,4


In [34]:
type(weatherpf['Hour'][0])

numpy.int64

In [33]:
print(df['Date'][0] == weatherpf['Date'][0] )
print(df['Hour'][0])

True
13


In [29]:
type(df['Date'][0])

datetime.date

In [28]:
# The weather sample is taken at the last minute of the hour, so the hour is one off 
weatherpf['Hour'] = (weatherpf['Hour'] + 1 )% 24

In [29]:
'''we want to combine the generation data to the weather data at the same time
we are going to predict the previous hours' weather to predict the next hour 
i.e. weather[hour_i] will try to predict generation[hour_i]
A BETTER WAY TO DO THIS:
set the date_hour as the index and then just join the two df's '''

"we want to combine the generation data to the weather data at the same time\nwe are going to predict the previous hours' weather to predict the next hour \ni.e. weather[hour_i] will try to predict generation[hour_i]\nA BETTER WAY TO DO THIS:\nset the date_hour as the index and then just join the two df's "

In [49]:
Generated = np.full((len(weatherpf),1), math.nan)

In [51]:
wCount = 0

for i in range(len(df)):
    added = False
    while not added:
        # if the date and hour match, we have found the right one
        if weatherpf['Date'][wCount] == df['Date'][i] and weatherpf['Hour'][wCount] == df['Hour'][i]:
            Generated[wCount] = df['Generated'][i]
            added = True
        else:
            wCount += 1 


KeyError: 0

In [48]:
# np.count_nonzero(~np.isnan(Generated))

121

In [38]:
weatherpf['Generated'] = Generated
weatherpf.head()

,date_time,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,td_avg,...,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel,Date,Hour,Generated
0,2019-07-11 00:59:59,30,0.002,0.003,0.0,0.741,0.0,31,29,36.9,...,4252,19.884,11.79,60.8,60.3,19.822,101.6,2019-07-11,1,NaN
1,2019-07-11 01:59:59,31,0.002,0.003,0.0,0.732,0.0,32,30,36.5,...,4263,19.884,11.79,58.5,58.3,19.822,101.6,2019-07-11,2,NaN
2,2019-07-11 02:59:59,32,0.001,0.002,0.0,0.716,0.0,33,30,36.0,...,4275,19.884,11.82,58.3,58.1,19.822,101.6,2019-07-11,3,NaN
3,2019-07-11 03:59:59,32,0.002,0.002,0.0,0.703,0.0,33,30,35.4,...,4241,19.635,11.78,57.6,57.4,19.822,101.6,2019-07-11,4,NaN
4,2019-07-11 04:59:59,32,0.001,0.002,0.0,0.686,0.0,33,30,34.9,...,4231,19.884,11.81,56.8,56.8,19.884,101.7,2019-07-11,5,NaN


In [43]:
for i in range(100):
    print(weatherpf.Generated[i])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
19005151.70371431
25139284.36885746
nan
20209068.581654288
15964916.304913778
10714623.438632196
5322336.015953261
1816007.9890391491
nan
nan
nan
nan
nan
nan
nan
nan
nan
1681200.5660589964
3973302.524345186
9012946.02997414
14015182.381352874
17966267.298056714
8664665.889970947
21747960.351551637
22071414.07435361
24864185.214993477
21592524.914652605
19607519.999050535
15686778.24166664
10605813.208911858
5250134.196785491
1864700.571576947
nan
nan
nan
nan
nan
nan
nan
nan
nan
1853040.6112185097
4014826.3162352913
3278973.4481548797
5610634.309121637
9056918.695416957
6599216.576928332
7601136.628109179
10885582.146870185
17292069.17369534
16931812.50405352
19508949.436129507
13839888.49089887
11986267.378496462
5253862.46722048
2714025.3759377976
nan
nan
nan
nan
nan
nan
nan
nan
nan
1682228.647266563
3908697.8396529537
8831608.289502168
14491549.781434398
18940879.042028986
21452808.4732284
20961733.576980956
20897161.443303645
24208615.

In [44]:
# # remove rows that don't have a generation value 
# weatherpf.dropna(axis=0, inplace=True)
# weatherpf.head()

,date_time,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,td_avg,...,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel,Date,Hour,Generated
12,2019-07-11 12:59:59,20,0.027,0.031,3.518,0.968,0.0,25,15,43.5,...,4511,17.771,12.55,92.1,95.0,18.765,101.6,2019-07-11,13,1.900515e+07
13,2019-07-11 13:59:59,18,0.029,0.033,3.472,0.848,0.0,20,15,40.3,...,4490,12.676,12.55,91.0,94.8,18.765,101.5,2019-07-11,14,2.513928e+07
15,2019-07-11 15:59:59,16,0.027,0.031,2.782,0.842,0.0,18,13,40.1,...,4470,19.884,12.55,86.7,93.9,19.014,101.4,2019-07-11,16,2.020907e+07
16,2019-07-11 16:59:59,16,0.022,0.026,2.202,0.782,0.0,19,13,38.1,...,4469,19.884,12.53,85.1,88.9,19.573,101.3,2019-07-11,17,1.596492e+07
17,2019-07-11 17:59:59,23,0.017,0.020,1.534,0.826,0.0,32,14,39.6,...,4399,19.884,12.23,83.5,78.1,19.014,101.3,2019-07-11,18,1.071462e+07


In [46]:
weatherpf.tail()

,date_time,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,td_avg,...,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel,Date,Hour,Generated
423,2019-07-28 15:59:59,27,0.022,0.025,2.660,1.048,0.0,34,20,45.7,...,4480,18.828,12.56,85.6,87.4,18.703,101.7,2019-07-28,16,1.243103e+07
424,2019-07-28 16:59:59,23,0.015,0.017,2.084,0.942,0.0,29,16,42.8,...,4481,14.913,12.46,87.6,88.9,18.890,101.6,2019-07-28,17,1.481631e+07
425,2019-07-28 17:59:59,23,0.015,0.017,1.414,0.821,0.0,33,14,39.4,...,4412,19.884,12.21,86.4,78.1,19.014,101.5,2019-07-28,18,1.012708e+07
426,2019-07-28 18:59:59,20,0.009,0.011,0.712,0.848,0.0,24,15,40.3,...,4390,19.822,12.13,76.3,69.4,19.325,101.5,2019-07-28,19,4.836637e+06
427,2019-07-28 19:59:59,22,0.004,0.005,0.115,0.794,0.0,24,20,38.5,...,4344,19.884,12.05,67.8,66.7,19.697,101.5,2019-07-28,20,1.733876e+06
